In [1]:
import os
import slp_data
import config


/home/shahsavari/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


split recordings to train test and validation sets

In [2]:
all_recs = []
for f in os.listdir(config.data_dir):
    if f.endswith('.lbl'):
        all_recs.append(f.split('.')[0])
all_recs = np.asarray(all_recs, dtype=np.str)

from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
for train_records_index, test_records_index in kf.split(all_recs):
    print("TRAIN:", train_records_index, "TEST:", test_records_index)
    train_recs = all_recs[train_records_index]
    test_recs = all_recs[test_records_index]

('TRAIN:', array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]), 'TEST:', array([0, 1]))


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
num_recs = len(all_recs)

train_recs = all_recs[:3*num_recs//5]
val_recs = all_recs[3*num_recs//5:4*num_recs//5]
test_recs = all_recs[4*num_recs//5:]

print("train recordings: {}".format(train_recs))
print("valid recordings: {}".format(val_recs))
print("test recordings: {}".format(test_recs))

data = slp_data.SlpDataSet(config.data_dir, 
                           train_recs,
                           val_recs, 
                           test_recs, one_hot=True)

In [3]:
# Training Parameters
num_steps = 1000
batch_size = 128
display_step = 10
strides = 1
#pooling kernel size and sride
k = 2

# Network Parameters
num_input = 30*250 #
num_classes = len(slp_data.DataSet.all_labels)  #
dropout = 0.2  # Dropout, probability to keep units

In [4]:
import tensorflow as tf

# tf Graph input
X = tf.placeholder(tf.float32,[None,num_input],name='image')
Y = tf.placeholder(tf.int32,[None,num_classes],name='label')
keep_prob = tf.placeholder(tf.float32)  # dropout (keep probability)

#your codes here
#  Store layers weight & bias
# The first two convolutional layer
w_c_1 = tf.Variable(tf.random_normal([50, 1, 1, 32], stddev=0.01))
w_c_2 = tf.Variable(tf.random_normal([50, 1, 32, 64], stddev=0.01))
b_c_1 = tf.Variable(tf.random_normal([32], stddev=0.1))
b_c_2 = tf.Variable(tf.random_normal([64], stddev=0.1))

# The second two convolutional layer weights
w_c_3 = tf.Variable(tf.random_normal([50, 1, 64, 64], stddev=0.01))
w_c_4 = tf.Variable(tf.random_normal([50, 1, 64, 64], stddev=0.01))
b_c_3 = tf.Variable(tf.random_normal([64], stddev=0.1))
b_c_4 = tf.Variable(tf.random_normal([64], stddev=0.1))

# Fully connected weight
w_f_1 = tf.Variable(tf.random_normal([125*15*64, 1024], stddev=0.01))
b_f_1 = tf.Variable(tf.random_normal([1024], stddev=0.1))

# output layer weight
w_out = tf.Variable(tf.random_normal([1024, num_classes], stddev=0.01))
b_out = tf.Variable(tf.random_normal([num_classes], stddev=0.01))

#
# Define model
x =tf.reshape(X,shape=[-1,250*30,1,1])
# first layer convolution
conv1 = tf.nn.conv2d(x, w_c_1, strides=[1, 1, 1, 1], padding='SAME')
conv1 = tf.add(conv1, b_c_1)
# second layer convolution
conv2 = tf.nn.conv2d(conv1, w_c_2, strides=[1, 1, 1, 1], padding='SAME')
conv2 = tf.add(conv2, b_c_2)
# first Max Pooling (down-sampling)
pool_1 = tf.nn.avg_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# third layer convolution
conv3 = tf.nn.conv2d(pool_1, w_c_3, strides=[1, 1, 1, 1], padding='SAME')
conv3 = tf.add(conv3, b_c_3)
# fourth layer convolution
conv4 = tf.nn.conv2d(conv3, w_c_4, strides=[1, 1, 1, 1], padding='SAME')
conv4 = tf.add(conv4, b_c_4)

# second Max Pooling (down-sampling)
pool_2 = tf.nn.avg_pool(conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# first Fully connected layer
# Reshape conv4 output to fit fully connected layer input and first fully connected layer
fc1 = tf.reshape(pool_2, [-1, w_f_1.get_shape().as_list()[0]])
fc1 = tf.nn.relu(tf.add(tf.matmul(fc1, w_f_1), b_f_1))
# Apply Dropout
fc1 = tf.nn.dropout(fc1,keep_prob)

In [5]:
#your codes here
# Output, class prediction
logits = tf.add(tf.matmul(fc1, w_out), b_out)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.argmax(prediction, axis=1)
y_true = tf.argmax(Y, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(correct_pred, y_true), tf.float32))
# accuracy, accuracy_op = tf.metrics.accuracy(labels=y_true, predictions=correct_pred)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
init = tf.global_variables_initializer()


# Start training
sess=tf.Session()
# Run the initializer
sess.run(init)
step = 1
epochs = 20
n_train_examples = data.train.get_examples_count()
total_steps = epochs * ( n_train_examples // batch_size )
for step in range(total_steps):
    batch_x, batch_y = data.train.next_batch(batch_size, shuffle=True)
    # Run optimization op (backprop)
    _ = sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
    if step % display_step == 0 or step == 1:
        # Calculate batch loss and accuracy
        loss,acc,pr,cpr,y_t=sess.run([loss_op,accuracy,prediction,correct_pred,y_true],feed_dict={X:batch_x,Y:batch_y,keep_prob:1.0})
        print("Step {:5d}/{} , Minibatch Loss={:12.4f} Training Accuracy={:7.4f}".format(step, total_steps, loss, acc))

    #print(cpr)
    #print(y_t)

print("Optimization Finished!")


recording   1/10 : ./MIT-BIH-true/slp59 with x,y shapes: (458, 7500) (458,)
recording   2/10 : ./MIT-BIH-true/slp02a with x,y shapes: (360, 7500) (360,)
recording   3/10 : ./MIT-BIH-true/slp14 with x,y shapes: (714, 7500) (714,)
recording   4/10 : ./MIT-BIH-true/slp32 with x,y shapes: (640, 7500) (640,)
recording   5/10 : ./MIT-BIH-true/slp16 with x,y shapes: (694, 7500) (694,)
recording   6/10 : ./MIT-BIH-true/slp48 with x,y shapes: (760, 7500) (760,)
recording   7/10 : ./MIT-BIH-true/slp01b with x,y shapes: (360, 7500) (360,)
recording   8/10 : ./MIT-BIH-true/slp67x with x,y shapes: (154, 7500) (154,)
recording   9/10 : ./MIT-BIH-true/slp41 with x,y shapes: (780, 7500) (780,)
recording  10/10 : ./MIT-BIH-true/slp03 with x,y shapes: (720, 7500) (720,)
x,y has been read with shape: (5640, 7500) (5640, 7)
Step     0/880 , Minibatch Loss=     37.7677 Training Accuracy= 0.2812
Step     1/880 , Minibatch Loss=    259.1886 Training Accuracy= 0.3359
Step    10/880 , Minibatch Loss=   1575.18

In [7]:
def print_measures(predicted_labels, true_labels):
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    cm = confusion_matrix(true_labels, predicted_labels)
    print(cm)
    print(classification_report(true_labels, predicted_labels))
    print("Accuracy : {}".format(accuracy_score(true_labels, predicted_labels)))
    
n_examples = data.test.get_examples_count()
import numpy as np
n_batches = n_examples // batch_size
predicted_labels = np.empty(shape=(0), dtype=np.int)
y_test_all = np.empty(shape=(0), dtype=np.int)

total_loss = 0.0
total_acc = 0.0
for i in range(n_batches):
    x_test, y_test = data.train.next_batch(batch_size, shuffle=False)
    loss,acc,pr,cpr,y_t=sess.run([loss_op,accuracy,prediction,correct_pred,y_true],feed_dict={X:x_test,Y:y_test,keep_prob:1.0})
    predicted_labels = np.concatenate((predicted_labels, cpr))
    y_test_all = np.concatenate((y_test_all, y_t))
    total_loss += loss
    total_acc += acc
    print("Batch {:5d}/{} , Minibatch Loss={:12.4f} Test Accuracy={:7.4f}".format(i, n_batches, loss, acc))
total_loss /= n_batches
total_acc /= n_batches
print("average loss: {} average accuracy: {}".format(total_loss, total_acc))

print_measures(predicted_labels, y_test_all[:len(predicted_labels)])

recording   1/4 : ./MIT-BIH-true/slp01a with x,y shapes: (240, 7500) (240,)
recording   2/4 : ./MIT-BIH-true/slp60 with x,y shapes: (710, 7500) (710,)
recording   3/4 : ./MIT-BIH-true/slp04 with x,y shapes: (720, 7500) (720,)
recording   4/4 : ./MIT-BIH-true/slp66 with x,y shapes: (439, 7500) (439,)
x,y has been read with shape: (2109, 7500) (2109, 7)
Batch     0/16 , Minibatch Loss=      0.2027 Test Accuracy= 0.9609
Batch     1/16 , Minibatch Loss=      0.2614 Test Accuracy= 0.9141
Batch     2/16 , Minibatch Loss=      0.2706 Test Accuracy= 0.9219
Batch     3/16 , Minibatch Loss=      0.2530 Test Accuracy= 0.9219
Batch     4/16 , Minibatch Loss=      0.2509 Test Accuracy= 0.9219
Batch     5/16 , Minibatch Loss=      0.2475 Test Accuracy= 0.9531
Batch     6/16 , Minibatch Loss=      0.2019 Test Accuracy= 0.9688
Batch     7/16 , Minibatch Loss=      0.2782 Test Accuracy= 0.9219
Batch     8/16 , Minibatch Loss=      0.2644 Test Accuracy= 0.9375
Batch     9/16 , Minibatch Loss=      0.201